In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import datetime

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

c:\users\berkan\appdata\local\programs\python\python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\berkan\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\users\berkan\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [28]:
pd.set_option('display.max_columns', 6)
df = pd.read_csv("2021-dublin-city-cycle-counts.csv")
df['day_of_week'] = df['Date & Time'].apply(lambda x: datetime.datetime.strptime(x, '%d-%m-%Y %H:%M:%S').strftime('%A'))
df['day_of_month'] = df['Date & Time'].apply(lambda x: datetime.datetime.strptime(x, '%d-%m-%Y %H:%M:%S').strftime('%d'))
df['day_of_week_num'] = df['Date & Time'].apply(lambda x: datetime.datetime.strptime(x, '%d-%m-%Y %H:%M:%S').strftime('%w'))
df['time_of_day'] = df['Date & Time'].apply(lambda x: datetime.datetime.strptime(x, '%d-%m-%Y %H:%M:%S').strftime('%H'))
df['month'] = df['Date & Time'].apply(lambda x: datetime.datetime.strptime(x, '%d-%m-%Y %H:%M:%S').strftime('%m'))
df = df.fillna(-1)

df.head(5)

,Date & Time,Aston Quay/Fitzgeralds,Aston Quay/Fitzgeralds IN,...,day_of_week_num,time_of_day,month
0,01-01-2021 00:00:00,165.0,65.0,...,5,00,01
1,01-01-2021 01:00:00,25.0,10.0,...,5,01,01
2,01-01-2021 02:00:00,27.0,12.0,...,5,02,01
3,01-01-2021 03:00:00,16.0,5.0,...,5,03,01
4,01-01-2021 04:00:00,16.0,8.0,...,5,04,01


In [21]:
def preprocessing(df):
    data = np.array(df)
    
    park_names = np.array(df.columns.values[1:-5])
    label_parks = data[:,1:-5]
    data_parks = data[:,-4:].astype(np.int64)
    
    len_parks = label_parks.shape[1]
    all_labels_parks = list()
    all_park_names = list()
    
    # Indices for days
#     monday_indices = np.where(data_parks[:,0] == 0)[0]
#     tuesday_indices = np.where(data_parks[:,0] == 1)[0]
#     wednesday_indices = np.where(data_parks[:,0] == 2)[0]
#     thursday_indices = np.where(data_parks[:,0] == 3)[0]
#     friday_indices = np.where(data_parks[:,0] == 4)[0]
#     saturday_indices = np.where(data_parks[:,0] == 5)[0]
#     sunday_indices = np.where(data_parks[:,0] == 6)[0]
    
    # Park Selection
    for i in range(len_parks):
        park = label_parks[:,i]
        indices_not_nan = np.where(park != -1)[0]
        indices_nan = np.where(park == -1)[0]
        if (len(indices_nan) < int( len(park) * 0.5)):

            for indice_nan in indices_nan:
                day = data_parks[indice_nan,0]
                time = data_parks[indice_nan,1]
                indices_mean = np.where((data_parks[:,0] == day) & (data_parks[:,1] == time))[0]
                park[indice_nan] = np.mean(park[indices_mean])
            all_labels_parks.append(park)
            all_park_names.append(park_names[i])
                
        
    return data_parks, np.array(all_labels_parks).reshape(label_parks.shape[0],-1), np.array(all_park_names)

In [22]:
# Data_parks columns = [day, hour, month]
# All_labels_parks columns = [Park1, Park2,.. etc]
data_parks, all_labels_parks, all_park_names = preprocessing(df)

In [23]:
df_train = pd.DataFrame(data_parks, columns=['day_of_month', 'day_of_week_num', 'time_of_day', 'month'])
df_label = pd.DataFrame(all_labels_parks, columns=all_park_names)

In [24]:
df_train.head(5)

,day_of_month,day_of_week_num,time_of_day,month
0,1,5,0,1
1,1,5,1,1
2,1,5,2,1
3,1,5,3,1
4,1,5,4,1


In [25]:
df_label.head(5)

,Bachelors walk/Bachelors way,Bachelors walk/Bachelors way IN,Bachelors walk/Bachelors way OUT,...,Westmoreland Street West/Carrolls,Westmoreland Street West/Carrolls IN,Westmoreland Street West/Carrolls OUT
0,66,18,11,...,159,91,259
1,206,166,101,...,117,110,87
2,15,10,36,...,62,106,109
3,74,86,90,...,28,44,9
4,31,4,35,...,75,52,60
